# This notebook prepares the data for NN models.
#### Clean the start and the end of each activity session
#### Select important columns and one hot encoding
#### Break down to shorter time periods
#### Encode output and shuffle the data


Data frame vocabulary:
- *df* = the raw csv dataframe
- *df_clean* = final cleaned and processed dataframe
- *df_level* = df for each activity level
- *df_level_clean* = clean version of df_level
- *df_temp* = a helper dtaframe to store temporary data for each participant and each level
    

In [1]:
import pandas as pd

In [2]:
input_dir =  'Z:/Research/dfuller/Walkabilly/studies/smarphone_accel/data/Ethica_Jaeger_Merged/pocket/'
input_file_name = 'pocket_with_couns_and_vec_meg_30Hz.csv'


In [3]:
df = pd.read_csv(input_dir + input_file_name)

Break it down to several CSV file, Lying, Sitting, Walking, Running3, Running5, Running7
Keep Gender, weight and height


In [13]:
participant_list = df.participant_id.unique()

In [ ]:
# select important columns, x, y, z, height, weight, age, gender, also participant_id for cleaning. remove it later
df = df['x_axis','y_axis','z_axis','participant_id','trimmed_activity','height','weight','age','gender']

In [39]:
# change gender to dummy
df.gender[df['gender']=='Female'] = 0
df.gender[df['gender']=='Male'] = 1

C:\Users\dfuller\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dfuller\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
df.head(3)

,record_time,x_axis,y_axis,z_axis,participant_id,wear_location,V.O2,VO2.kg,V.CO2,V.E,trimmed_activity,height,weight,age,gender,x,y,z,counts_vec_mag
0,2019-01-07T10:30:01Z,0.266111,-0.069464,0.966880,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,0,13.0,0.0,73.0,74.1485
1,2019-01-07T10:30:01.033300Z,0.266401,-0.069351,0.963832,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,0,NaN,NaN,NaN,74.1485
2,2019-01-07T10:30:01.066700Z,0.262441,-0.067394,0.943335,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,0,NaN,NaN,NaN,74.1485


In [6]:
#df_L = df[df['trimmed_activity'] == 'Lying']

##### Get rid of the first and last 20 Seconds for avoiding noise

In [ ]:
# n_ignore = 600 # ignores 20 sec with a frequency of 30 Hz

# for partip in participant_list:
#     df_temp = df_L[df_L['participant_id'] == partip]
#     df_temp_nrow = df_temp.shape[0]
#     df_temp = df_temp[n_ignore:df_temp_nrow-n_ignore]
#     df_sub_clean = pd.concat(df_sub_clean, df_temp)

# # the first row of sub is zeros    
# df_L = df_sub_clean[1:,]
# df_L = df_L.drop('participant_id', axis = 1)

    

In [54]:
# list(df.columns)

In [52]:
# repeat for all PE levels 

# get levels to loop thru
PE_levels = df.trimmed_activity.unique()

# see dataframe vocabulary
# ceate empty df
df_clean = df.iloc[0,].copy()
df_clean = x.replace(df_clean,0)

for level in PE_levels:
    df_level = df[df['trimmed_activity'] == level]
    df_level_clean = df.iloc[0,].copy()
    df_level_clean = x.replace(df_level_clean,0)

    
    n_ignore = 600 # ignores 20 sec with a frequency of 30 Hz

    for partip in participant_list:
        df_temp = df_level[df_level['participant_id'] == partip]
        df_temp_nrow = df_temp.shape[0]
        df_temp = df_temp[n_ignore:df_temp_nrow-n_ignore]
        df_level_clean = pd.concat(df_level_clean, df_temp)

    # the first row of this df is zeros    
    df_level_clean = df_level_clean[1:,]
    df_clean = pd.concat(df_clean, df_level_clean)

# the first row of this df is zeros    
df_clean = df_clean[1:,]  
df_clean = df_clean.drop('participant_id', axis = 1)
    

In [17]:
# break into 150(use a variable) readings chunks by adding a helper column
# spread based on the helper column
# bind all the files of different levels
# export as the final output

In [18]:
# before training
# suffle
# Seperate to x and y (y is the levels)
# one hot encode y